<a href="https://colab.research.google.com/github/souza-eab/EO/blob/main/Babi_2024_Asset_biomass_ref_maps_Version1_OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1- Login - set your project in GCP

In [ ]:
import ee
import os

In [ ]:
import pandas as pd
from google.cloud import storage

#2 - Autenticar chaves

In [ ]:
!earthengine authenticate --auth_mode=gcloud

Authenticate: Limited support in Colab. Use ee.Authenticate() or --auth_mode=notebook instead.
W0520 18:17:32.113934 132456755265536 _default.py:683] No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
Fetching credentials using gcloud
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&state=ntNFpO7fU0KGkwLjkpIvXaKP9i3iJu&prompt=consent&token_usage=remote&access_type=offline&code_challenge=nQ4XeTdS5sWziJ1IGrxR08TDXWdhIRXVwacvZmK3fCY&code_challenge_method=S256

Once finish

#3 - Setar projeto para cobrança

In [ ]:
!gcloud auth application-default set-quota-project hwp-qcn


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "hwp-qcn" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [ ]:
#Conferir
from pprint import pprint
ee.Authenticate()

True

# 4 - Quais arquivos estam na pasta da GCP

In [ ]:
from google.cloud import storage

def list_files_in_bucket(bucket_name, project_id):
    """
    Lists all files in the specified bucket.

    Args:
        bucket_name: Name of the Google Cloud Storage bucket.
        project_id: ID of your Google Cloud project.

    Returns:
        List of file names.
    """
    # Initialize the Google Cloud Storage client with the project ID
    storage_client = storage.Client(project=project_id)

    # Get the bucket
    bucket = storage_client.get_bucket(bucket_name)

    # List all files in the bucket
    blobs = bucket.list_blobs()

    # Collect file names
    file_names = [blob.name for blob in blobs]

    return file_names

# Specify the name of your bucket and your project ID
bucket_name = 'gcp_qcn'
project_id = 'hwp-qcn'

# List all files in the bucket
files_in_bucket = list_files_in_bucket(bucket_name, project_id)

# Print the names of all files in the bucket
for file_name in files_in_bucket:
    print(file_name)

PDigital2000_2023_AMZ_raster.tif


In [ ]:
def prepare_data_for_upload(files_in_bucket, bucket_name):
    """
    Prepares data for upload by creating URLs and extracting base names.

    Args:
        files_in_bucket: List of file names in the bucket.
        bucket_name: Name of the Google Cloud Storage bucket.

    Returns:
        List of Google Cloud Storage URLs, base names, and Google Earth Engine asset paths.
    """
    gcp_files = []
    base_names = []
    gee_assets = []

    for file_name in files_in_bucket:
        # Construct GCP file URL
        gcp_file_url = f"gs://{bucket_name}/{file_name}"
        gcp_files.append(gcp_file_url)

        # Extract base name
        base_name = file_name.split('/')[-1].split('.')[0]
        base_names.append(base_name)

        # Construct GEE asset path
        gee_asset = f"projects/mapbiomas-workspace/CCAL/DATASETS/PRODES/{base_name}"
        gee_assets.append(gee_asset)

    return gcp_files, base_names, gee_assets


# 5 - Verificar os caminhos da GCP, nome_base que vai para o GEE e no .EE Assets paths


In [ ]:
# Function
gcp_files, base_names, gee_assets = prepare_data_for_upload(files_in_bucket, bucket_name)

# Prints
print("Google Cloud Storage URLs:")
for url in gcp_files:
    print(url)

print("\nBase Names:")
for name in base_names:
    print(name)

print("\nGoogle Earth Engine Asset Paths:")
for asset in gee_assets:
    print(asset)

Google Cloud Storage URLs:
gs://gcp_qcn/PDigital2000_2023_AMZ_raster.tif

Base Names:
PDigital2000_2023_AMZ_raster

Google Earth Engine Asset Paths:
projects/mapbiomas-workspace/CCAL/DATASETS/PRODES/PDigital2000_2023_AMZ_raster


# Verificar dados

In [ ]:
pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 53.4 MB/s eta 0:00:00


In [ ]:
import rasterio

In [ ]:
from google.cloud import storage
project_id = 'hwp-qcn'  # ID do projeto
client = storage.Client(project=project_id)


### Tipo de dados

In [ ]:
import os
import rasterio
from google.cloud import storage

# ID do projeto
project_id = 'hwp-qcn'

# Inicializar o cliente do Google Cloud Storage
client = storage.Client(project=project_id)

# Nome do bucket
bucket_name = 'gcp_qcn'

# Caminhos para os arquivos no bucket
gcp_file_urls = [
   # 'gs://gcp_qcn/AGB_NVmasked_RF-v0.tif',
   # 'gs://gcp_qcn/AGB_NVmasked_RF_cog-v0.tif'
]

# Baixar e processar cada arquivo
for gcp_file_url in gcp_file_urls:
    # Obter o nome do arquivo a partir da URL
    file_name = gcp_file_url.split('/')[-1]

    # Baixar o arquivo do bucket
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    download_path = f'/tmp/{file_name}'
    blob.download_to_filename(download_path)

    # Processar o arquivo usando rasterio
    with rasterio.open(download_path) as src:
        # Exemplo: obter metadados da imagem
        print(src.profile)
        # Exemplo: acessar as bandas da imagem
        for band in src.indexes:
            band_data = src.read(band)
            print(band_data)

    # Remover o arquivo temporário
    os.remove(download_path)


#6 - Export from .EE

In [ ]:
import subprocess
from google.cloud import storage

def upload_files_to_earth_engine(bucket_name, asset_name, project_id):
    """
    Uploads files from a Google Cloud Storage bucket to Google Earth Engine.

    Args:
        bucket_name (str): Name of the Google Cloud Storage bucket.
        asset_name (str): Name of the asset in Google Earth Engine.
        project_id (str): ID of your Google Cloud project.

    Returns:
        None
    """
    # Initialize the Google Cloud Storage client with the project ID
    storage_client = storage.Client(project=project_id)

    # Get the bucket
    bucket = storage_client.get_bucket(bucket_name)

    # Create empty lists to store file information
    gcp_files = []
    base_names = []
    gee_assets = []

    # List all blobs in the bucket and process them
    for blob in bucket.list_blobs():
        # Build the Google Cloud Storage URL
        gcp_file = f'gs://{bucket_name}/{blob.name}'
        gcp_files.append(gcp_file)

        # Build the base name
        base_name = f'{asset_name}/{blob.name.split("/")[-1]}'
        base_names.append(base_name)

        # Build the Google Earth Engine asset path
        gee_asset = f'{asset_name}/{blob.name.split("/")[-1].split(".")[0]}'
        gee_assets.append(gee_asset)

    # Perform batch upload of files to Google Earth Engine
    for i, gcp_file in enumerate(gcp_files):
        print(f'Ingesting file: {gcp_file}')
        print(f'File {i + 1} of {len(gcp_files)} ({round((i + 1) / len(gcp_files) * 100)}%)')

        # Upload the file to Google Earth Engine
        subprocess.run(['earthengine', 'upload', 'image', '--asset_id=' + gee_assets[i],'--nodata_value=0',  gcp_file])
        print('')

    print('Upload completed.')

# Path assets:
bucket_name = 'gcp_qcn'
asset_name = 'projects/mapbiomas-workspace/CCAL/DATASETS/PRODES'
project_id = 'hwp-qcn'
upload_files_to_earth_engine(bucket_name, asset_name, project_id)


Ingesting file: gs://gcp_qcn/PDigital2000_2023_AMZ_raster.tif
File 1 of 1 (100%)

Upload completed.
